In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio
import os
import matplotlib.pyplot as plt
from ipywidgets import interact
from keras.layers import Input, Dense, Dropout,Conv2D,MaxPooling2D,Flatten
from keras.layers import GlobalMaxPooling2D,UpSampling2D,GlobalMaxPooling1D
from keras.models import Model
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Print out the folders where our datasets live. 
print("Datasets: {0}".format(os.listdir("../input/danburyaijuneworkshop/")))

In [7]:
train_x = np.load("../input/train_x.npy")
train_y = np.load("train_y.npy")
train_y = train_y - 1
test_x  = np.load("test.npy")

In [ ]:
def makeModel(inputSize):
    inputs = Input(shape=inputSize,name="input")
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')(x)
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = GlobalMaxPooling2D()(x)
    x = Dense(100,activation='relu')(x)
    out = Dense(10,activation='softmax', name="output")(x)

    model = Model(inputs=inputs, outputs=out)
    model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['acc'])
    
    return model

def learningCurves(hist):
    histLoss_train      = hist.history['loss']
    epochs = len(histLoss_train)
    
    plt.figure(figsize=(12,12))
    
    plt.plot(range(epochs),histLoss_train, label="Training Loss", color="#acc6ef")


    plt.xlabel('Epochs',fontsize=14)
    plt.title("Learning Curves",fontsize=20)

    plt.legend()
    plt.show()

In [ ]:
model2 = makeModel((32,32,3,))
model2.summary()
hist2 = model2.fit(train_x, to_categorical(train_y,10),batch_size=100,epochs=500)

In [ ]:
learningCurves(hist2)

In [ ]:
pred = model2.predict(test_x)
pred = np.argmax(pred,1)
pred = pred + 1

In [ ]:
submission = pd.DataFrame.from_items([
    ('id',list(range(pred.shape[0]))),
    ('label', pred)])

submission.to_csv('submission.csv', index = False)